In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
from torch.autograd import Variable
import os
import torch.optim as optim
import pickle
import sys
import pdb

# Pandas dataset by us

In [ ]:
with open('../git_repo/Code-Generation/pickles/pandas_comments_filtered_data.pkl', 'rb') as fp:
    pandas_data = pickle.load(fp)

In [ ]:
raw_data = []
for item in pandas_data:
    raw_data.extend(item[0])

# Conala data 

In [2]:
fp = open('../conala-baseline/conala-corpus/conala-train.snippet', 'r')
raw_data = fp.readlines()
fp.close()

# create dictionaries 

In [6]:
vocabulary = set()
for line in raw_data:
    chars = list(line)
    vocabulary = vocabulary.union(set(chars))

In [7]:
len(vocabulary)

140

In [8]:
char2id = dict()
id2char = dict()
for i, char in enumerate(vocabulary):
    char2id[char] = i
    id2char[i] = char

# Data Loading

In [9]:
def sample_gumbel(shape, eps=1e-10, out=None):
    """
    Sample from Gumbel(0, 1)
    based on
    https://github.com/ericjang/gumbel-softmax/blob/3c8584924603869e90ca74ac20a6a03d99a91ef9/Categorical%20VAE.ipynb ,
    (MIT license)
    """
    U = out.resize_(shape).uniform_() if out is not None else torch.rand(shape)
    return - torch.log(eps - torch.log(U + eps))

In [10]:
def preprocess_inputs(data):
    converted_data = []
    for line in data:
        x = [char2id[char] for char in list(line)]
        converted_data.extend(x)
    return np.array(converted_data)

In [11]:
total_data = preprocess_inputs(raw_data)

In [12]:
class TextDataLoader(DataLoader):

    def __init__(self, data, batch_size=1):
        self.data = data
        self.batch_size = batch_size

    def __iter__(self):
        data = preprocess_inputs(self.data)
        n = len(data) - 1
        m = n // self.batch_size
        data = data[:self.batch_size * m + 1]
        inputs = data[:-1].reshape((self.batch_size, m)).T
        targets = data[1:].reshape((self.batch_size, m)).T

        pos = 0

        while n - pos > 0:

            l = np.random.random_integers(40, 60)
            if pos + l >= m:
                break

            yield inputs[pos:pos+l], targets[pos: pos+l]
            pos += l


In [13]:
N = len(raw_data)
permute = np.random.permutation(N)
raw_data = np.array(raw_data)[permute]
idx = int(0.7* len(raw_data))
train_data = raw_data[0:idx]
dev_data = raw_data[idx:]

In [14]:
train_loader = TextDataLoader(train_data, batch_size=32)

In [15]:
vocabsize = len(vocabulary)

# Model 

In [16]:
class Net(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2word = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence, forward):

        embeds = self.word_embeddings(sentence)
        lstm_out, h = self.lstm1(embeds)
        h = self.hidden2word(lstm_out)

        gumbel = Variable(sample_gumbel(shape=h.size(), out=h.data.new()))
        h += gumbel
        logits = h

        if forward > 0:
            outputs = []
            logits = torch.transpose(logits, 0, 1)
            h = torch.max(logits[:, -1:, :], dim = 2)[1].t()

            for i in range(forward):
                h = self.word_embeddings(h)
                h, _ = self.lstm1(h)
                h = self.hidden2word(h)
                gumbel = Variable(sample_gumbel(shape=h.size(), out=h.data.new()))
                h += gumbel
                outputs.append(h)
                h = torch.max(h, dim=2)[1]

            logits = torch.transpose(logits, 0, 1)
            logits = torch.cat([logits] + outputs, dim=0)
            logits = torch.max(logits, dim=2)[1]
        return logits


In [17]:
model = Net(64, 128, vocabsize, vocabsize)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
model.cuda()

Net(
  (word_embeddings): Embedding(140, 64)
  (lstm1): LSTM(64, 128)
  (hidden2word): Linear(in_features=128, out_features=140, bias=True)
)

In [24]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)
for epoch in range(10):
    for i, data in enumerate(train_loader):
        inputs, targets = data
        inputs, targets = Variable(torch.LongTensor(inputs).cuda()), Variable(torch.LongTensor(targets).cuda())

        model.zero_grad()

        word_scores = model(inputs, 0)

        r, c, h = word_scores.shape
        word_scores = word_scores.view(r * c, h)

        targets = targets.contiguous().view(-1)

        loss = criterion(word_scores, targets)
        print('Loss is ', loss.data.cpu()[0])
        loss.backward()
        optimizer.step()

/home/spothara/anaconda3/envs/i_study/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: This function is deprecated. Please call randint(40, 60 + 1) instead
/home/spothara/anaconda3/envs/i_study/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Loss is  tensor(2.0086)
Loss is  tensor(1.9969)
Loss is  tensor(1.9119)
Loss is  tensor(2.0448)
Loss is  tensor(1.8843)
Loss is  tensor(1.9232)
Loss is  tensor(1.7868)
Loss is  tensor(1.8638)
Loss is  tensor(1.8641)
Loss is  tensor(1.7699)
Loss is  tensor(2.0594)
Loss is  tensor(1.9251)
Loss is  tensor(1.9632)
Loss is  tensor(1.8480)
Loss is  tensor(2.0721)
Loss is  tensor(1.9549)
Loss is  tensor(1.8769)
Loss is  tensor(2.0491)
Loss is  tensor(1.9199)
Loss is  tensor(1.9387)
Loss is  tensor(1.8780)
Loss is  tensor(1.8070)
Loss is  tensor(1.8182)
Loss is  tensor(1.8770)
Loss is  tensor(1.8209)
Loss is  tensor(1.9493)
Loss is  tensor(1.8177)
Loss is  tensor(1.8053)
Loss is  tensor(1.9434)
Loss is  tensor(1.9372)
Loss is  tensor(1.9668)
Loss is  tensor(1.9344)
Loss is  tensor(1.9326)
Loss is  tensor(1.8888)
Loss is  tensor(1.9609)
Loss is  tensor(1.8245)
Loss is  tensor(1.8183)
Loss is  tensor(1.8848)
Loss is  tensor(1.8421)
Loss is  tensor(2.0029)
Loss is  tensor(1.8265)
Loss is  tensor(

Loss is  tensor(2.0625)
Loss is  tensor(1.8411)
Loss is  tensor(1.9758)
Loss is  tensor(1.8429)
Loss is  tensor(2.0302)
Loss is  tensor(1.9547)
Loss is  tensor(1.8533)
Loss is  tensor(1.9445)
Loss is  tensor(1.9587)
Loss is  tensor(2.0349)
Loss is  tensor(1.7965)
Loss is  tensor(1.9328)
Loss is  tensor(1.8791)
Loss is  tensor(1.8292)
Loss is  tensor(1.8869)
Loss is  tensor(1.8685)
Loss is  tensor(1.8721)
Loss is  tensor(1.7244)
Loss is  tensor(1.8605)
Loss is  tensor(1.9448)
Loss is  tensor(1.9174)
Loss is  tensor(1.9746)
Loss is  tensor(1.9734)
Loss is  tensor(1.7599)
Loss is  tensor(1.9536)
Loss is  tensor(1.8413)
Loss is  tensor(1.8393)
Loss is  tensor(1.9608)
Loss is  tensor(1.8276)
Loss is  tensor(1.8525)
Loss is  tensor(2.0109)
Loss is  tensor(1.7813)
Loss is  tensor(1.7074)
Loss is  tensor(2.0178)
Loss is  tensor(2.0296)
Loss is  tensor(1.8118)
Loss is  tensor(1.9516)
Loss is  tensor(1.8272)
Loss is  tensor(1.8826)
Loss is  tensor(1.8988)
Loss is  tensor(1.9725)
Loss is  tensor(

# Evaluation 

In [25]:
dev_loader = TextDataLoader(dev_data, batch_size=32)

In [26]:
model.train(False)

Net(
  (word_embeddings): Embedding(140, 64)
  (lstm1): LSTM(64, 128)
  (hidden2word): Linear(in_features=128, out_features=140, bias=True)
)

In [27]:
final_ip = []
final_op = []
for epoch in range(1):
    for i, data in enumerate(dev_loader):
        inputs, targets = data
        final_ip.append(inputs.transpose())
        inputs, targets = Variable(torch.LongTensor(inputs).cuda()), Variable(torch.LongTensor(targets).cuda())

        word_scores = model(inputs, 20)
        outputs = word_scores.transpose(0, 1)
        
        final_op.append(outputs[:, -20:].cpu().data.numpy())
        

/home/spothara/anaconda3/envs/i_study/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: This function is deprecated. Please call randint(40, 60 + 1) instead


In [28]:
final_ip[0].shape, final_op[0].shape

((32, 43), (32, 20))

In [29]:
fp = open('dummy_conala_train.txt', 'w')
for i, item in enumerate(final_ip):
    for j, line in enumerate(item):
        chars = [id2char[x] for x in line]
        fp.write(''.join(chars))
        op_chars = [id2char[x] for x in final_op[i][j]]
        fp.write(''.join(chars))
        fp.write('\n')